In [ ]:
import torch
import numpy as np
import pandas as pd
from torch.nn.utils.rnn import PackedSequence

In [3]:
embeding_shapes = {}
TENSOR_IDS = {}

In [4]:
class LstmLayer(): pass
class PackedSequenceBatchNorm1d(): pass
class PackedSequenceEmbedingLayer(): pass
class EmbedingLayer(): pass
def cat_packed_seq(): pass

In [ ]:
class NN(torch.Module):
    def __init__(self, feats_4_model, emb_shapes, lstm_sizes, drop=0.3):
        super().__init__()

        self.lstm_sizes = lstm_sizes

        embeding_output_size = {}
        for key in embeding_shapes.keys():
            embeding_output_size[key] = sum(v[-1] for v in embeding_shapes[key])

        #### 3D

        c_tens_id = "TENSOR_3D"
        eq_3D_num_feats = feats_4_model[c_tens_id]["numerical"]
        c_input_size = embeding_output_size[c_tens_id] + len(eq_3D_num_feats)

        self.eq_3d_lstm = LstmLayer(c_input_size, **lstm_sizes[TENSOR_IDS["EQ_3D"]])
        self.eq_3d_bn = PackedSequenceBatchNorm1d(len(eq_3D_num_feats))
        self.eq_3d_emb = PackedSequenceEmbedingLayer(emb_shapes[c_tens_id])
        ########

        ##### 2D
        c_tens_id = "TENSOR_2D"
        c_num_feats = feats_4_model[c_tens_id]["numerical"]
        c_input_size = (
            embeding_output_size[c_tens_id]
            + len(c_num_feats)
            + lstm_sizes[TENSOR_IDS["EQ_3D"]["hidden_state"]]
        )
        self.eq_2d_lstm = LstmLayer(c_input_size, **lstm_sizes[TENSOR_IDS["EQ_2D"]])
        self.eq_2d_bn = PackedSequenceBatchNorm1d(len(c_num_feats))
        self.eq_2d_emb = PackedSequenceEmbedingLayer(emb_shapes[c_tens_id])
        #####

        #### 1D
        eq_1D_num_feats = feats_4_model["TENSOR_1D"]["numerical"]
        sum_req_num_feats = feats_4_model["headers"]

        # MODIFICATION
        concated_numerical_1d_data_size = len(eq_1D_num_feats) + len(sum_req_num_feats)
        linear1_input_size = concated_numerical_1d_data_size + embeding_output_size['ONE_DIM']
        linear1_input_size += lstm_sizes[TENSOR_IDS['EQ_2D']]['hidden_state']

        self.fc_branc = torch.nn.Sequential(
            torch.nn.Dropout(drop),
            torch.nn.Linear(linear1_input_size, 1)
        )

        self.bn1 = torch.nn.BatchNorm1d(
            concated_numerical_1d_data_size
        )
        self.emb1 = EmbedingLayer(emb_shapes['ONE_DIM'])
        ########

    def forward_3d(self, tensor_3d):
        flatten_3d_num, flatten_3d_emb, emb_3d_lens = tensor_3d
        flatten_3d_num = self.eq_3d_bn(flatten_3d_num)

        lstm_out = self.eq_3d_lstm(flatten_3d_num)

        out = torch.split(lstm_out, emb_3d_lens)
        out = torch.nn.utils.rnn.pack_sequence(out, enforce_sorted=False)
        return out

    def forward_2d(self, tensor_2d, tensor_3d_lstm_out):
        tensor_2d_num, tensor_2d_emb = tensor_2d

        num = self.eq_2d_bn(tensor_2d_num)

        emb = self.eq_2d_emb(tensor_2d_emb)

        data = cat_packed_seq([num, emb, tensor_3d_lstm_out])
        return self.eq_2d_lstm(data)

    def forward_1d(self, one_dim_data, eq_branch_out):
        numerical_1d, embedings_1d = one_dim_data
        numerical_1d = self.bn1(numerical_1d)
        embedings_1d = self.emb1(embedings_1d)
        fc = torch.cat(
            (numerical_1d, embedings_1d, eq_branch_out),
            axis = -1
        )

        logits = self.fc_branc(fc)

        return logits
    
    @staticmethod
    def preproc_input(ONE_DIM, TENSOR_2D, TENSOR_3D):
        return ONE_DIM, TENSOR_2D, TENSOR_3D
    
    def forward(self, **kwargs):
        tensor_1d, tensor_2d, tensor_3d = self.preproc_input(**kwargs)

        data_3d_out = self.forward_3d(tensor_3d)

        data_2d_out = self.forward_2d(tensor_2d, data_3d_out)

        logits = self.forward_1d(tensor_1d, data_2d_out)

        return logits


# мое

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NNWithoutLSTM(nn.Module):
    def __init__(self, feats_4_model, emb_shapes, lstm_sizes, drop=0.3):
        super().__init__()

        self.lstm_sizes = lstm_sizes

        # Считаем размер выхода эмбеддингов
        embeding_output_size = {}
        for key in emb_shapes.keys():
            embeding_output_size[key] = sum(v[-1] for v in emb_shapes[key])

        #### 3D - Замена LSTM на полносвязный слой
        c_tens_id = "TENSOR_3D"
        eq_3D_num_feats = feats_4_model[c_tens_id]["numerical"]
        c_input_size = embeding_output_size[c_tens_id] + len(eq_3D_num_feats)

        self.eq_3d_fc = nn.Linear(c_input_size, lstm_sizes["EQ_3D"]["hidden_state"])
        self.eq_3d_bn = nn.BatchNorm1d(len(eq_3D_num_feats))
        self.eq_3d_emb = nn.EmbeddingBag(embeding_output_size[c_tens_id], lstm_sizes["EQ_3D"]["hidden_state"], mode='mean')
        ########

        ##### 2D - Замена LSTM на полносвязный слой
        c_tens_id = "TENSOR_2D"
        c_num_feats = feats_4_model[c_tens_id]["numerical"]
        c_input_size = (
            embeding_output_size[c_tens_id]
            + len(c_num_feats)
            + lstm_sizes["EQ_3D"]["hidden_state"]  # Передаем скрытое состояние от 3D ветки
        )

        self.eq_2d_fc = nn.Linear(c_input_size, lstm_sizes["EQ_2D"]["hidden_state"])
        self.eq_2d_bn = nn.BatchNorm1d(len(c_num_feats))
        self.eq_2d_emb = nn.EmbeddingBag(embeding_output_size[c_tens_id], lstm_sizes["EQ_2D"]["hidden_state"], mode='mean')
        #####

        #### 1D - Полносвязная ветка
        eq_1D_num_feats = feats_4_model["TENSOR_1D"]["numerical"]
        sum_req_num_feats = feats_4_model["headers"]

        # Расчет входных данных для 1D ветки
        concated_numerical_1d_data_size = len(eq_1D_num_feats) + len(sum_req_num_feats)
        linear1_input_size = concated_numerical_1d_data_size + embeding_output_size['ONE_DIM']
        linear1_input_size += lstm_sizes["EQ_2D"]["hidden_state"]

        self.fc_branc = nn.Sequential(
            nn.Dropout(drop),
            nn.Linear(linear1_input_size, 1)
        )

        self.bn1 = nn.BatchNorm1d(concated_numerical_1d_data_size)
        self.emb1 = nn.EmbeddingBag(embeding_output_size['ONE_DIM'], lstm_sizes['EQ_2D']['hidden_state'], mode='mean')
        ########

    def forward_3d(self, tensor_3d):
        flatten_3d_num, flatten_3d_emb, emb_3d_lens = tensor_3d
        flatten_3d_num = self.eq_3d_bn(flatten_3d_num)

        # Полносвязный слой вместо LSTM
        fc_out = F.relu(self.eq_3d_fc(flatten_3d_num))
        return fc_out

    def forward_2d(self, tensor_2d, tensor_3d_fc_out):
        tensor_2d_num, tensor_2d_emb = tensor_2d

        num = self.eq_2d_bn(tensor_2d_num)
        emb = self.eq_2d_emb(tensor_2d_emb)

        # Объединяем и передаем через полносвязный слой
        data = torch.cat([num, emb, tensor_3d_fc_out], dim=-1)
        fc_out = F.relu(self.eq_2d_fc(data))
        return fc_out

    def forward_1d(self, one_dim_data, eq_branch_out):
        numerical_1d, embedings_1d = one_dim_data
        numerical_1d = self.bn1(numerical_1d)
        embedings_1d = self.emb1(embedings_1d)

        # Объединяем и передаем через полносвязную ветку
        fc = torch.cat((numerical_1d, embedings_1d, eq_branch_out), axis=-1)
        logits = self.fc_branc(fc)
        return logits

    @staticmethod
    def preproc_input(ONE_DIM, TENSOR_2D, TENSOR_3D):
        return ONE_DIM, TENSOR_2D, TENSOR_3D

    def forward(self, **kwargs):
        tensor_1d, tensor_2d, tensor_3d = self.preproc_input(**kwargs)

        # Обрабатываем данные по 3D, 2D и 1D веткам
        data_3d_out = self.forward_3d(tensor_3d)
        data_2d_out = self.forward_2d(tensor_2d, data_3d_out)
        logits = self.forward_1d(tensor_1d, data_2d_out)

        return logits


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NNWithConvLSTM(nn.Module):
    def __init__(self, feats_4_model, emb_shapes, lstm_sizes, drop=0.3):
        super().__init__()

        self.lstm_sizes = lstm_sizes

        # Считаем размер выхода эмбеддингов
        embeding_output_size = {}
        for key in emb_shapes.keys():
            embeding_output_size[key] = sum(v[-1] for v in emb_shapes[key])

        #### 3D - Смешивание LSTM и Conv1d
        c_tens_id = "TENSOR_3D"
        eq_3D_num_feats = feats_4_model[c_tens_id]["numerical"]
        c_input_size = embeding_output_size[c_tens_id] + len(eq_3D_num_feats)

        self.eq_3d_lstm = LstmLayer(c_input_size, **lstm_sizes["EQ_3D"])
        self.eq_3d_bn = PackedSequenceBatchNorm1d(len(eq_3D_num_feats))
        self.eq_3d_emb = PackedSequenceEmbedingLayer(emb_shapes[c_tens_id])

        # Новый сверточный слой для обработки числовых данных
        self.eq_3d_conv = nn.Conv1d(in_channels=len(eq_3D_num_feats), out_channels=32, kernel_size=3, padding=1)
        self.eq_3d_conv_bn = nn.BatchNorm1d(32)
        ########

        ##### 2D - Смешивание LSTM и Conv1d
        c_tens_id = "TENSOR_2D"
        c_num_feats = feats_4_model[c_tens_id]["numerical"]
        c_input_size = (
            embeding_output_size[c_tens_id]
            + len(c_num_feats)
            + lstm_sizes["EQ_3D"]["hidden_state"]  # Передаем скрытое состояние от 3D ветки
        )

        self.eq_2d_lstm = LstmLayer(c_input_size, **lstm_sizes["EQ_2D"])
        self.eq_2d_bn = PackedSequenceBatchNorm1d(len(c_num_feats))
        self.eq_2d_emb = PackedSequenceEmbedingLayer(emb_shapes[c_tens_id])

        # Новый сверточный слой для обработки числовых данных
        self.eq_2d_conv = nn.Conv1d(in_channels=len(c_num_feats), out_channels=32, kernel_size=3, padding=1)
        self.eq_2d_conv_bn = nn.BatchNorm1d(32)
        #####

        #### 1D
        eq_1D_num_feats = feats_4_model["TENSOR_1D"]["numerical"]
        sum_req_num_feats = feats_4_model["headers"]

        # Расчет входных данных для 1D ветки
        concated_numerical_1d_data_size = len(eq_1D_num_feats) + len(sum_req_num_feats)
        linear1_input_size = concated_numerical_1d_data_size + embeding_output_size['ONE_DIM']
        linear1_input_size += lstm_sizes["EQ_2D"]["hidden_state"]

        self.fc_branc = nn.Sequential(
            nn.Dropout(drop),
            nn.Linear(linear1_input_size, 1)
        )

        self.bn1 = nn.BatchNorm1d(concated_numerical_1d_data_size)
        self.emb1 = EmbedingLayer(emb_shapes['ONE_DIM'])
        ########

    def forward_3d(self, tensor_3d):
        flatten_3d_num, flatten_3d_emb, emb_3d_lens = tensor_3d

        # Обрабатываем числовые данные через сверточный слой
        flatten_3d_num = flatten_3d_num.unsqueeze(1)  # Добавляем размер для Conv1d
        flatten_3d_num = F.relu(self.eq_3d_conv_bn(self.eq_3d_conv(flatten_3d_num)))
        flatten_3d_num = flatten_3d_num.squeeze(1)  # Убираем добавленный размер
        flatten_3d_num = self.eq_3d_bn(flatten_3d_num)

        # Используем LSTM для обработки эмбеддингов
        lstm_out = self.eq_3d_lstm(flatten_3d_num)
        out = torch.split(lstm_out, emb_3d_lens)
        out = torch.nn.utils.rnn.pack_sequence(out, enforce_sorted=False)
        return out

    def forward_2d(self, tensor_2d, tensor_3d_lstm_out):
        tensor_2d_num, tensor_2d_emb = tensor_2d

        # Обрабатываем числовые данные через сверточный слой
        tensor_2d_num = tensor_2d_num.unsqueeze(1)  # Добавляем размер для Conv1d
        tensor_2d_num = F.relu(self.eq_2d_conv_bn(self.eq_2d_conv(tensor_2d_num)))
        tensor_2d_num = tensor_2d_num.squeeze(1)  # Убираем добавленный размер
        num = self.eq_2d_bn(tensor_2d_num)

        emb = self.eq_2d_emb(tensor_2d_emb)

        # Объединяем данные и передаем через LSTM
        data = cat_packed_seq([num, emb, tensor_3d_lstm_out])
        return self.eq_2d_lstm(data)

    def forward_1d(self, one_dim_data, eq_branch_out):
        numerical_1d, embedings_1d = one_dim_data
        numerical_1d = self.bn1(numerical_1d)
        embedings_1d = self.emb1(embedings_1d)

        # Объединяем данные и передаем в полносвязную ветку
        fc = torch.cat(
            (numerical_1d, embedings_1d, eq_branch_out),
            axis=-1
        )

        logits = self.fc_branc(fc)

        return logits

    @staticmethod
    def preproc_input(ONE_DIM, TENSOR_2D, TENSOR_3D):
        return ONE_DIM, TENSOR_2D, TENSOR_3D

    def forward(self, **kwargs):
        tensor_1d, tensor_2d, tensor_3d = self.preproc_input(**kwargs)

        # Обрабатываем данные по 3D, 2D и 1D веткам
        data_3d_out = self.forward_3d(tensor_3d)
        data_2d_out = self.forward_2d(tensor_2d, data_3d_out)
        logits = self.forward_1d(tensor_1d, data_2d_out)

        return logits
